In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import networkx.algorithms.community as nx_comm

In [4]:
#make sure pandas understand that columns are filled with list
import ast
cc = pd.read_csv("../data/creditcard.csv")
edges = pd.read_csv('../data/data_nodes_l2.csv')
edges['name_50'] = edges['name_50'].apply(lambda x: ast.literal_eval(x))
edges['dist_50'] = edges['dist_50'].apply(lambda x: ast.literal_eval(x))

In [5]:
g_part= nx.read_gpickle("../data/pickles2/myGraph" + str(0) + ".gpickle")

In [6]:
#create communities
comms = nx_comm.louvain_communities(g_part, seed =1)
len(comms)

18

In [81]:
#check for similar communities
from tqdm import tqdm_notebook
#create df to fill

results_list = []


for x in tqdm_notebook(range(0,1)):
    perct_comms = []
    comms = nx_comm.louvain_communities(g_part, seed =x)

    df = pd.DataFrame(index=range(len(g_part)),columns=['community'])
    #fill with current communities
    comm_len = len(comms)
    for comm in range(0, comm_len):
        for i in comms[comm]:
            df.community[i] = comm

    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    for t in tqdm_notebook((range(1,48))):
        start = edges.time.searchsorted(t, side='left')
        end = edges.time.searchsorted(t, side='right')
        com_chosen = []




        df_total_com = df.groupby(['community'])['community'].count()
        df['fraud'] = cc.Class
        df_fraud_coms = df[df.fraud ==1]
        df_fraud_coms = df_fraud_coms.groupby(['community']).count()
        result = pd.concat([df_total_com, df_fraud_coms], axis=1)
        result['percentage'] = np.divide(result.fraud, result.community, out=np.zeros_like(result.fraud), where=result.community!=0)
        result['percentage'] = result['percentage'].fillna(0)




        for i in range(start,  end):
            namelist = edges.name_50[i]

            com_list = []

            for n in namelist:
                try:
                    com_list.append(df.community[n])
                except:
                    com_list.append(float("NaN"))
            com_list_nona = [item for item in com_list if not(pd.isnull(item)) == True]
            try:
                most_freq_com = np.bincount(com_list_nona).argmax()
            except:
                most_freq_com = float("NaN")
            com_chosen.append(most_freq_com)


            try:
                perct_comms.append( result['percentage'][most_freq_com])
            except:
                perct_comms.append(float("NaN"))




        df = df.append(pd.DataFrame(com_chosen, columns=['community']), ignore_index=True)

    df_total_com = df.groupby(['community'])['community'].count()
    df['fraud'] = cc.Class
    df_fraud = df[df.fraud ==1]
    df_fraud_coms = df_fraud.groupby(['community']).count()
    result = pd.concat([df_total_com, df_fraud_coms], axis=1)
    results_list.append(result)

C:\Users\joost\AppData\Local\Temp\ipykernel_9696\3291560104.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x in tqdm_notebook(range(0,1)):


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\joost\AppData\Local\Temp\ipykernel_9696\3291560104.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for t in tqdm_notebook((range(1,48))):


  0%|          | 0/47 [00:00<?, ?it/s]

In [114]:
lst = [0] * len(g_part)
total_list = lst+perct_comms

data_v7 = cc.copy()
data_v7['com_pct'] = total_list
data_v7['com_pct'] = data_v7['com_pct'].fillna(0)
data_v7.to_csv('../data/final_data/creditcard_v7.csv', index=False)

In [109]:
data_v7


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,com_pct
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0.000000
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,0.000000
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,0.000000
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.000000
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0,0.001133
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0,0.000000
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0,0.000000
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0,0.002999


In [107]:
len(total_list)

284807

In [60]:
result

,community,fraud,percentage
community,,,
0,53,NaN,0.000000
1,259,NaN,0.000000
2,189,NaN,0.000000
3,128,NaN,0.000000
4,267,NaN,0.000000
5,302,NaN,0.000000
6,60,NaN,0.000000
7,454,NaN,0.000000
8,44,1.0,0.022727


,community,fraud,percentage
community,,,
0,53,NaN,0.000000
1,259,NaN,0.000000
2,189,NaN,0.000000
3,128,NaN,0.000000
4,267,NaN,0.000000
5,302,NaN,0.000000
6,60,NaN,0.000000
7,454,NaN,0.000000
8,44,1.0,0.022727


In [19]:
df_fraud

,community,fraud
541,8,1
623,16,1
4920,7.0,1
6108,17.0,1
6329,8.0,1
...,...,...
279863,14.0,1
280143,14.0,1
280149,14.0,1
281144,14.0,1


In [ ]:
total_results = pd.concat(results_list, axis = 1, sort=False)

In [ ]:
total_results.to_csv('../data/percentage_comms.csv', index=False)